# Trabajo práctico 7 - Contornos

**Alumnos:**

- Carol lugones Ignacio (100073)
- Torresetti Lisandro (99846)

## Objetivo

Sobre la imagen de los bloques de la semana pasada encontrar los mismos y obtener los parámetros de área, perímetro y orientación. Adicionalmente indicar la relación de aspecto (largo Vs. ancho) de cada uno. Comparar los datos obtenidos obtenidos para cada bloque entre sí.

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
%matplotlib inline

In [ ]:
def plotter(image, title = '', imgSize = (18,9), grayScale = False, step = 100): #Funcion auxiliar para realizar los graficos
    plt.figure(figsize=imgSize)
    plt.title(title, fontsize = 16, fontweight = "bold")
    plt.imshow(image) if not grayScale else plt.imshow(image, cmap='gray', vmin=0, vmax=255)
    plt.yticks(np.arange(0, len(image), step))
    plt.xticks(np.arange(0, len(image[0]), step), rotation=90)
    plt.show()  

In [ ]:
#Cargamos la imagen a analizar
img = cv.imread('bloques1.jpg')
imgGray = cv.imread('bloques1.jpg', 0)
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
#img = cv.medianBlur(img,5) #Para eliminar los posibles ruidos
imgWidth = img.shape[1]
imgHeight = img.shape[0]

print("Img Width: {} \t Img Height: {}".format(imgWidth, imgHeight))
plotter(img, 'Original Image')
plotter(imgGray, 'Original Image Grayscale', grayScale=True)

In [ ]:
#Extraemos la parte que nos interesa analizar, o sea la tabla verde con los bloques
imgCut = img[800:3300, 200:2400, :]
imgCutGray = imgGray[800:3300, 200:2400]
plotter(imgCut, 'Image Cutted')
plotter(imgCutGray, 'Image Cutted Gray Scale', grayScale=True)

A diferencia del tp anterior, en este caso no vamos a tomar en cuenta el bloque que se encuentra tapado por los otros bloques, así que sus valores de media y desvío no se analizarán.

In [ ]:
imgPrueba = img.copy()
# Bloques numerados de arriba hacia abajo y de izq a derecha
#El tercer valor es el ancho del bloque para hacer la mascara
block1 = ((550, 980), (1350, 980), 345)
block2 = ((1500, 1600), (2050, 1600),125)
block3 = ((650, 2400), (1000, 2400), 750)
block4 = ((1300, 2400), (1600, 2400), 700)
block6 = ((1890, 2200), (2250, 2200),800)
blocks = [block1, block2, block3, block4, block6]

# Marcamos una linea roja para ver correctamente sus posiciones
for block in blocks:
    cv.line(imgPrueba, block[0], block[1], (255,0,0), 5)

plotter(imgPrueba, step = 100)

In [ ]:
def createMask(img, samples, lowerMultiplier = 15, upperMultiplier = 6):
    meanColors = 0
    stdColors = 0
    for point1, point2, blockWidth in samples:
        colorMean, colorStd = cv.meanStdDev(img[point1[1]:point1[1] + blockWidth,point1[0]:point2[0], :])
        meanColors += colorMean
        stdColors += colorStd
    meanColors /= len(samples)
    stdColors /= len(samples)
    return cv.inRange(img, meanColors - stdColors * lowerMultiplier,  meanColors + stdColors * upperMultiplier)

In [ ]:
mask = createMask(img, blocks, 7)
imgWithMask = cv.bitwise_and(img, img, mask=mask)
plotter(imgWithMask, 'Image With Mask' ,step = 100)

In [ ]:
imgWithMaskGray = cv.cvtColor(imgWithMask, cv.COLOR_RGB2GRAY)
imgWithMaskGray = imgWithMaskGray[800:3300, 200:2400]
plotter(imgWithMaskGray,'Gray Image With Mask', grayScale=True, step = 100)

In [ ]:
#Aplicamos un filtro de mediana a las dos imagenes para eliminar los ruidos
imgCutGray = cv.medianBlur(imgCutGray, 5)
imgWithMaskGray = cv.medianBlur(imgWithMaskGray, 5)
plotter(imgCutGray, 'Cut',grayScale=True)
plotter(imgWithMaskGray, 'Mask',grayScale=True)

In [ ]:
IMGS = [imgCutGray, imgWithMaskGray] #Estas son las imagenes que vamos a analizar
NAMES = ['Original', 'With Mask']

Binarizamos todas las imágenes con el algoritmo de Otsu, ya que las operaciones morfológicas se aplican sobre imágenes binarias. Antes de hacer esto primero graficamos los histogramas para analizar mejor las imágenes

In [ ]:
def plotHistograms(imgs, bins = 50):
    fig, axs = plt.subplots(len(imgs))
    fig.suptitle('Histograms', fontsize=18, fontweight='bold')
    for imgNum,img in enumerate(imgs):
        axs[imgNum].set_title(NAMES[imgNum], fontsize = 16, fontweight='bold')
        axs[imgNum].grid()
        axs[imgNum].hist(img.ravel(),bins,[0,256], color='orange')
    
    fig.set_size_inches(10, 8)
    fig.tight_layout(pad=3.0)

In [ ]:
plotHistograms([imgCutGray, imgWithMaskGray])

Observamos que los valores de threshold de 127 funcionará bien para ambas imágenes. Este valor lo ponemos como parámetro opcional en la siguiente función.

In [ ]:
def otsuBinarization(imgs, thresh = 127):
    result = []
    for imgNum, img in enumerate(imgs):
        ret, imgBin = cv.threshold(img,thresh,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
        plotter(imgBin,NAMES[imgNum], grayScale=True)
        result.append(imgBin)
    return result

In [ ]:
imgBin, imgMaskBin = otsuBinarization([imgCutGray, imgWithMaskGray])

Se pueden apreciar diferencias entre ambas binarizaciones. Dado que la opción con la máscara eliminó de mejor manera el bloque tapado, a partir de ahora utilizaremos esta imagen para realizar las operaciones y los análisis correspondientes.

A continuación realizaremos varias operaciones morfológicas para ver los resultados de cada una, así luego las refinamos para eliminar efectivamente el bloque que se encuentra tapado.

In [ ]:
#PRUEBAS DE OPERACIONES MORFOLOGICAS
imgTest = imgMaskBin.copy()

# Erode
kernelErode = np.ones((9,10), np.uint8)
plotter(cv.erode(imgTest, kernelErode), 'Erode', grayScale=True)

# Dilate
kernelDilate =np.ones((15,15), np.uint8)
plotter(cv.dilate(imgTest, kernelDilate), 'Dilate', grayScale=True)

# Opening
kernelOpening = np.ones((30,30), np.uint8)
plotter(cv.morphologyEx(imgTest, cv.MORPH_OPEN, kernelOpening),  'Opening', grayScale=True)

# Closing
kernelClosing = np.ones((30,30), np.uint8)
plotter(cv.morphologyEx(imgTest, cv.MORPH_CLOSE, kernelClosing), 'Closing', grayScale=True)

# Multiple Erosions and Dilations

kernelErotion2 = np.ones((15,15), np.uint8)
kernelDilation2 = np.ones((15,10), np.uint8)
imgTest2 = imgTest.copy()

imgTest2 = cv.erode(imgTest2, kernelErotion2, iterations=2)
imgTest2 = cv.dilate(imgTest2, kernelDilation2, iterations=5)
plotter(imgTest2, 'Multiple Operations', grayScale=True)


De las operaciones anteriores la que mejor deja los bordes de los bloques es la dilatación, buscaremos mejorar el kernel para conseguir un mejor resultado.

In [ ]:
# TESTING
testing = imgMaskBin.copy()
#kernelErode = np.ones((10,1), np.uint8)
#testing = cv.erode(testing, kernelErode, iterations = 5)
kernelDilate = np.ones((2,4), np.uint8)
testing = cv.dilate(testing, kernelDilate, iterations=2)
plotter(testing, 'Dilate Modified', grayScale=True)

In [ ]:
imgDilated = cv.dilate(imgMaskBin.copy(), kernelDilate, iterations=2)
contours, hier = cv.findContours(imgDilated, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
out = imgCut.copy()
cv.drawContours(out, contours, -1, (255,0,0),5)
plotter(out)

Para eliminar los contornos no deseados, como los puntos rojos o curvas raras que se ven, vamos a calcular el área y desechar los contornos que tengan un valor menor que la media.

In [ ]:
def getAspectRatio(contour):
    x,y,width,height = cv.boundingRect(contour)
    return float(width) / height

def getCentroid(moment, area):
    return (int(moment['m10'] / area), int(moment['m01'] / area))

def getOrientation(contour):
    return cv.fitEllipse(contour)

In [ ]:
# Para mas info https://en.wikipedia.org/wiki/Image_moment
AREA = 0 #Para saber en que posicion del array que guarda el diccionario se encuentra la info deseada
PERIMETER = 1
CENTER = 2
ASPECT_RELATION = 3
ORIENTATION = 4
CONTOUR = 5

def getContoursInfo(contours):
    contoursArea = []
    contoursPerimeter = []
    contoursCenter = []

    for ctr in contours:
        moment = cv.moments(ctr)
        area = moment['m00']
        contoursArea.append(area)
        contoursPerimeter.append(cv.arcLength(ctr,True))
        contoursCenter.append(getCentroid(moment, area))

    contoursAreaMean = np.mean(contoursArea)

    contoursInfo = {}  
    contourNum = 1
    for area, perimeter, center, contour in  zip(contoursArea, contoursPerimeter, contoursCenter, contours):
        if area >= contoursAreaMean:
            contoursInfo[contourNum] = [area, perimeter, center, getAspectRatio(contour),
                                        getOrientation(contour), contour]
            contourNum += 1
            
    return contoursInfo

In [ ]:
contoursInfo = getContoursInfo(contours)

def printResults(contoursInfo):
    print('RESULTS')
    for key, value in contoursInfo.items():
        print('''
                Block Num: {}

                    Area: {}
                    Perimeter: {}
                    Centroid: {}
                    Aspect Relation: {}
                    Orientation: {}
                '''.format(key, value[AREA], value[PERIMETER], value[CENTER], value[ASPECT_RELATION],
                          value[ORIENTATION][2]))
    

Para saber que bloque posee esos valores los numeraremos en la siguiente imagen, además agregamos un punto blanco que muestra donde se encuentra el centro de bloque.

In [ ]:
out = imgCut.copy()
font = cv.FONT_HERSHEY_SIMPLEX
for key, value in contoursInfo.items():
    contour = value[CONTOUR]
    cv.putText(out, str(key), value[CENTER], font, 5, (0, 0, 0), 10, cv.LINE_AA)
    cv.circle(out, value[CENTER], radius=15, color=(255, 255, 255), thickness=-3)
    approx = cv.approxPolyDP(contour, 0.1 * cv.arcLength(contour, True), True)
    cv.drawContours(out, [approx], 0, (255, 0, 0), 10)
    x = approx.ravel()[0]
    y = approx.ravel()[1] - 5
plotter(out)

In [ ]:
# Dibujamos las elipese que se obtuvieron al calcular la orientacion mas arriba
out = imgCut.copy()
font = cv.FONT_HERSHEY_SIMPLEX
for key, value in contoursInfo.items():
    contour = value[CONTOUR]
    center, axes, angle = value[ORIENTATION]
    center = (int(center[0]), int(center[1]))
    axes = (int(axes[0]), int(axes[1]))
    cv.putText(out, str(key), value[CENTER], font, 5, (0, 0, 0), 10, cv.LINE_AA)
    approx = cv.approxPolyDP(contour, 0.1 * cv.arcLength(contour, True), True)
    #cv2.ellipse(image, center, axes, angle, 0., 360, (0,0,255))
    
    cv.ellipse(out,center,axes,angle,0.,360,(0,0,0), 5)
    cv.drawContours(out, [approx], 0, (255, 0, 0), 10)
    x = approx.ravel()[0]
    y = approx.ravel()[1] - 5
plotter(out)

In [ ]:
printResults(contoursInfo)

## Conclusiones

El bloque con mayor área es el número 3, esto se puede deber a que el bloque se encuentra más arriba que los otros por lo que esta más cerca del punto en el cual se tomó la foto. Al ser el que posee el área más grande se esperaba que también su perímetro lo fuera.

Con respecto a los algoritmos utilizados se pudo apreciar lo eficiente y simples que son las operaciones morfológicas para restaurar partes de la imagen (cuando los bloques tenían puntos negros en su interior o las esquinas no estaban bien definidas) o borrarlas (como fue el caso del bloque que se encontraba tapado que se lo descartó).

Otra cosa que llamó la atención es toda la información que se puede obtener a partir de los contornos.